## Veri Setinin Hazırlanması 

Veri Seti ve Adresi:

Animation_Movies = https://www.kaggle.com/datasets/harits/anime-database-2022

In [ ]:
# Kullanacağımız kütüphaneleri yükledim.

# Pandas ve NumPy
import pandas as pd
import numpy as np

# Görselleştirme kütüphaneleri
import matplotlib.pyplot as plt
import seaborn as sns

# Diğer örnek görselleştirme kütüphaneleri
# ggplot
# Bokeh
# Plotly

In [ ]:
# Veri setini notebook içerisine dahil ettim.

df_ = pd.read_csv(filepath_or_buffer = "/kaggle/input/anime-database-2022/Anime.csv")
df = df_.copy()

In [ ]:
# Veri setinde rasgele eksik veriler oluşturdum.

import random

def add_random_missing_values(dataframe: pd.DataFrame,
                              missing_rate: float = 0.05,
                              seed: random = 42) -> pd.DataFrame:
    """Turns random values to NaN in a DataFrame.
    
    To use this function, you need to import pandas, numpy and random libraries.

    Args:
        dataframe (pd.DataFrame): DataFrame to be processed.
        missing_rate (float): Percentage of missing value rate in float format. Defaults 0.05

    
    """
    # Get copy of dataframe
    df_missing = dataframe.copy()

    # Obtain size of dataframe and number total number of missing values
    df_size = dataframe.size
    num_missing = int(df_size * missing_rate)
    
    # Set seed
    if seed:
        random.seed(seed)

    # Get random row and column indexes to turn them NaN
    for _ in range(num_missing):
        row_idx = random.randint(0, dataframe.shape[0] - 1)
        col_idx = random.randint(0, dataframe.shape[1] - 1)

        df_missing.iat[row_idx, col_idx] = np.nan
        
    return df_missing

df = add_random_missing_values(dataframe = df,
                               missing_rate = 0.03)

## 1. Veriye İlk Bakış

In [ ]:
# Veri setinin ilk 3 gözlemini gözlemledim.
df.head(n = 3)

# Sondan 3 gözlem için tail() metodu kullanılabilir.
# df.tail(3)

Veri setindeki satır ve sütun sayılarını .shape kullanarak görebiliyoruz.

In [ ]:
df.shape # -> (satır, sütun)

Veri setindeki değişkenlerin isimlerine erişmek istersek df.columns kullanabiliriz.

In [ ]:
# Veri setindeki kolonları yazdırır.
print(list(df.columns), '\n')

# len() fonksiyonu ile toplam kolon sayısına erişiyoruz.
print(f"Veri seti içerisinde toplam {len(df.columns)} kolon vardır.\n")

# Çoğu zaman kullandığımız attribute ve fonksiyonların sonuçlarının hangi tipte veri döndürdüğünü gözlemlemek kod üzerinde hakimiyet kurmak için önemlidir.
print(f"df.columns'un döndüğü veri tipi: {type(df.columns)}\n")

info() metodunu kullanarak, veri setine ait yapısal bilgilere ulaşabiliriz.

Bu noktada elimizde kolonların veri tiplerinin doğru olup olmadığını, kayıt sayılarını, kolon isimlerini mutlaka kontrol etmeliyiz.

In [ ]:
df.info()

Sadece elimizdeki değişkenleri görmek ve tiplerini görüntülemek istersek, dtypes kullanabiliriz.

In [ ]:
df.dtypes

Veri setimizde hem kategorik hem de sayısal özellikler bulunmaktadır. Bu özellikleri ayırarak farklı değişken listelerinde kullanmak, özellikle kolon bazında işlem yapmayı planladığımızda faydalı olacaktır. Böylece bu listeleri, gerektiğinde filtre olarak kullanabiliriz.

Dikkat!: Veri setinin ham haliyle kolonların veri tipleri her zaman beklenen şekilde olmayabilir. Kategorik ve sayısal kolonları ayırmadan önce her bir değişkenin doğru veri tipiyle tanımlandığından emin olmalısınız.z.

In [ ]:
categorical_features = []
numerical_features = []

for col in df.columns:
    if (df[col].dtype == "object") or (df[col].dtype == "categorical"):
        categorical_features.append(col)
    else:
        numerical_features.append(col)

# ALIŞTIRMA: List comprehension nasıl kullanılır öğrenin ve yukarıdaki listeleri bu yapıyı kullanarak oluşturmaya çalışın.

In [ ]:
categorical_features, numerical_features

Örneğin, veri tipi "Object" olan kategorik değişkenleri "Categorical" olarak dönüştürmek istiyorsak, kolon isimleri üzerinde bir döngü oluşturarak bu işlemi kolayca gerçekleştirebiliriz.

Döngüde, eğer mevcut kolon ismi "categorical_features" listesinde yer alıyorsa, bu kolonun veri tipini "Categorical" olarak değiştirebiliriz; eğer listede bulunmuyorsa, herhangi bir işlem yapmaya gerek yoktur.m.

In [ ]:
for col in df.columns:
    if col in categorical_features:
        df[col] = pd.Categorical(df[col])

In [ ]:
df.info()

In [ ]:
df['Genres'][:3]

Veri setimde olan betimsel istatistiklerine ulaşmak için describe() fonksiyonunu kullandım.

Bu fonksiyon eksik verileri göz ardı eder ve default olarak yalnızca numerik değişkenlere bmaktadırar.

In [ ]:
df.describe().T # -> .T eklentisi elde edeceğimiz çıktıyı transpose eder.

# df.describe(include = "all") -> Veri setindeki tüm değişkenleri dahil etmek için include parametresine "all" değerini verebilirsiniz.

## 2. Eksik Veri Analizi

Eksik veri analizi ile, veri setimizdeki eksik değerleri belirleyip bu durumun uygun şekilde çözülmesini sağlamayı amaçlarız.

Veri setindeki eksik değerler, yapısal bir problemin belirtisi olabilir ve bu durum mutlaka doğru yöntemlerle ele alınmalıdır.

Eksik veriler, duruma göre veri setinden çıkarılabilir veya uygun değerlerle doldurulabilir. Ancak eksik verilerin silinmesi, silinen satır veya sütunlardaki diğer verilerin kaybına yol açabilir. Doldurma işlemi yapılırsa, veri setine yeni sentetik değerler ekleneceğinden, bu durum veri dağılımlarını etkileyebilir ve yanlılık yaratabilir.

Eksik verilerin neden oluştuğu dikkatlice incelenmeli ve bu tespitten sonra eksik verilerle nasıl başa çıkılacağına karar verilmelidir.lidir.

**2.1 Eksik Verilerin Gözlemlenmesi**

In [ ]:
df.isna().sum() # -> isna() methodu yerine isnull() methodu da kullanılabilir.

# ALIŞTIRMA: Her bir değişken için, o değişkendeki eksik değer sayısının toplam kayıt sayısının % kaçı olduğunu bulunuz.

Veri setindeki toplam eksik değer sayısını görmek için tekrar sum() fonksiyonu ile ekledim.

In [ ]:
df.isna().sum().sum()

In [ ]:
# Eksik olmayan değerlerin sayısı
df.notnull().sum()

In [ ]:
# Veri setinde toplam kaç adet eksik gözlem var, kaç adet eksik olmayan gözlem var görelim.
print(f"Veri seti içerisinde toplam {df.notnull().sum().sum()} adet eksik olmayan, {df.isnull().sum().sum()} eksik gözlem var.")

In [ ]:
# Veri setinde en az bir gözlemi eksik olan kayıtlara da ulaşabiliriz.
df[df.isnull().any(axis = 1)]

In [ ]:
# Hiç eksik gözlemi bulunmayan kayıtları getirelim.
df[df.notnull().all(axis = 1)][:5]

In [ ]:
#!pip install missingno -> missingno kütüphanesini kullanabilmek için öncelikle yüklemeniz lazım.
import missingno as msno

msno.bar(df = df,
         figsize = (8, 4),
         fontsize = 10);

In [ ]:
# msno.matrix(), değişkenlerde bulunan eksik değerlerin ilişkili olup olmadığını görsel yolla tespit etmek için kullanılabilir.
msno.matrix(df = df[['Type', 'Producers']],
            figsize = (10, 6),
            fontsize = 10);

In [ ]:
# Nullity Correlation (Heatmap)
# 1'e ne kadar yakınsa, ilişki ihtimali o kadar yüksek.
# 0 ise, birbirlerini etkileyen bir durum yoktur.
msno.heatmap(df = df,
             figsize = (10, 6),
             fontsize = 10);

**2.2 Yöntem 1: Eksik Verilerin Silinmesi**

Veri setindeki eksik verilere müdahale etmek için kullanılan yöntemlerden biri, eksik verilerin silinmesidir. Bu yöntem uygulaması bakımından oldukça basit olsa da, eksik verileri silmeden önce göz önünde bulundurulması gereken bazı önemli noktalar vardır.

1. Bir gözlemde eksik veri bulunduğunda, bu eksikliğin doğal olmayan bir nedenle meydana geldiğinden emin olmamız gerekir. Örneğin, elektrikli araçlar için motor hacmi kolonunda NaN değeri bulunması, doğal bir eksikliktir. Bu tür durumlarda, veriyi silmek yerine, eksik veriyi uygun bir yöntemle doldurmak daha doğru olabilir.


2. Eğer eksik veriler veri setinin büyük bir kısmını oluşturuyorsa, bu verilerin silinmesi halinde çok sayıda gözlem kaybedeceğimizi unutmamalıyız. Bu durumda, veri setindeki önemli bilgileri de kaybetmiş oluruz. Veri miktarının analitik ve makine öğrenmesi yöntemleri için kritik öneme sahip olduğunu göz önünde bulundurarak, veri setinden olabildiğince az kayıp olacak yöntemler tercih edilmelidir.

In [ ]:
# Eksik verilerin dropna ile silinmesi.
# Kalıcı bir değişiklik yapmaz, bunu yapmak için inplace argümanı kullanılmalı veya atama yapılmalıdır.

df.dropna(inplace = False)[:5]
#df = df.dropna()

In [ ]:
# Sadece bütün değerleri eksik olan bir gözlemi silmek istersek;
df.dropna(how = 'all')[:5]

In [ ]:
# Değişken bazında silmek için;
df.dropna(axis = 1)

**2.3 Yöntem 2: Eksik Verilerin Doldurulması**

Eksik verilerin doldurulması kararı, silme işlemi gibi dikkatlice ve bilinçli bir şekilde değerlendirilmesi gereken bir adımdır. Çünkü bu işlem, veri setinde gürültüye (noise) yol açabilir ve verinin istatistiksel güvenilirliğini olumsuz etkileyebilir. Ayrıca, analitik süreçlerde yanlış sonuçlara neden olabilir. Bu nedenle, eksik verileri doldurma kararı verildiğinde, her zaman potansiyel yanlılık risklerinin de göz önünde bulundurulması gerekmektedir.

In [ ]:
# Doldurma işlemlerini gerçekleştirmek için veri setimin bir kopyasını oluşturuyorum.
df_fillna = df.copy()

df_fillna.head(3)

In [ ]:
df_fillna.isna().sum()

**2.3.1 Sayısal Değişkenlerin Doldurulması**

In [ ]:
# Numerik bir değişkenin mean değeriyle doldurulması
mean_salary = df_fillna['Score'].mean()

df_fillna['Score'].fillna(value = mean_salary, inplace = True)

df_fillna.isna().sum()

In [ ]:
# Doldurma işleminin döngüyle yapılması
to_be_filled = numerical_features[1:]

for col in to_be_filled:
    df_fillna[col].fillna(df[col].mean(), inplace = True)

In [ ]:
df_fillna.isna().sum()

In [ ]:
df.describe().T

In [ ]:
# Bir sayısal değişkenin dağılımını görmek için histogram kullanabiliriz.
plt.hist(df['Score'])
plt.show()

**2.3.2 Kategorik Değişkenlerin Doldurulması**

In [ ]:
# Kategorik bir değişkenin mode değeriyle doldurulması

# Broadcast değişkeninde toplam kaç adet eksik değer olduğunu görelim.
df_fillna['Broadcast'].isna().sum()

#Broadcast değişkeninde en çok tekrar eden değeri(mode) alalım.
Broadcast_mode = df_fillna['Broadcast'].mode()

# Broadcast değişkenini mode değeri ile dolduralım.
df_fillna['Broadcast'] = df_fillna['Broadcast'].fillna(value = Broadcast_mode[0])

# Broadcast değişkeninde toplam kaç adet eksik değer olduğunu tekrar görelim.
df_fillna.isna().sum()

In [ ]:
df_fillna[["Type"]][:20]

In [ ]:
# Önceki değer ile doldurma işlemi
df_fillna["Type"].fillna(method = "bfill")

In [ ]:
# Sonraki değer ile doldurma işlemi
df_fillna["Type"].fillna(method = "ffill")

**2.3.3 Kategorik Kırılım İle Doldurma İşlemi**

Burada basit bir şekilde ortalama (mean) ve medyan (median) değerlerle eksik verileri doldurmuş olsak da, her durumda bu tür basit yöntemler doğru olmayabilir. Bu tür doldurma yöntemleri yalnızca hızlı bir çözüm olarak kullanılmalıdır. Daha analitik bir yaklaşım benimsemek için, veri setinde benzerlikler (imputation) kullanarak eksik verileri daha uygun şekilde tamamlayabiliriz.


Örneğin, salary kolonundaki eksik verileri ortalama ile doldurduk. Ancak elimizde job_title adında farklı pozisyonları içeren bir kolon olduğunu biliyoruz. Bu durumda, Data Engineer pozisyonundaki bir eksik maaş verisini, aynı pozisyondaki diğer maaşların ortalama (veya medyan) değeriyle doldurmak daha mantıklı olacaktır. Çünkü farklı pozisyonlar arasında maaş seviyeleri değişiklik gösterir; dolayısıyla eksik veriyi, ait olduğu pozisyonun istatistiksel değeriyle doldurmak daha doğru bir yaklaşım olacaktır.r.

In [ ]:
df_categorical_fillna = df.copy()

df_categorical_fillna.isna().sum()


Bu kod, bir veri çerçevesindeki (df_categorical_fillna) verileri ID sütununa göre gruplar ve her grup için Score sütununun ortalama değerini hesaplar. Gruplama işlemi, ID sütununun benzersiz değerlerine göre yapılır ve her grup için Score değerlerinin ortalaması alınır. Elde edilen sonuç, bir veri çerçevesine (DataFrame) dönüştürülerek ID_mean_Score_df değişkenine atanır. Sonuç, her ID için bir satır ve o gruba ait Score ortalamalarını içeren bir tablo oluşturur. Eğer ID sütunu kategorik bir veri türündeyse, observed=False parametresi tüm olası kategorileri hesaba katar, ancak kategorik değilse bu parametre etkisizdir.

In [ ]:
ID_mean_Score_df = df_categorical_fillna.groupby("ID", observed = False)["Score"].mean().to_frame()

In [ ]:
df_categorical_fillna["Score"].fillna(value = df_categorical_fillna.groupby("ID")["Score"].transform("mean"),
                                       inplace = True)

In [ ]:
df_categorical_fillna.isna().sum()

In [ ]:
df_categorical_fillna[df_categorical_fillna[["ID", "Score"]].isna().all(axis=1)][["ID", "Score"]]

In [ ]:
df_categorical_fillna[df_categorical_fillna['Score'].isnull()]

**2.3.4 Makine Öğrenmesi ile Değer Atama Teknikleri**

Makine öğrenmesi yöntemleri kullanarak da eksik verileri doldurmak mümkündür. Makine Öğrenmesi modelleri bu bootcamp'in konusu olmadığı için detaylı bir anlatım gerçekleştirilmeyecektir.

**Hangi yöntemler kullanılabilir ?**

* KNNImputer (K-Nearest Neighbor)

* Random Forest Classifier

## 3. Kategorik Değişken Analizi

In [ ]:
df = df_.copy()
df.info()

**3.1 Kategorik Değişken İşlemleri**

In [ ]:
# Spesifik bir veri tipi tutan değişkenleri seçmek için select_dtypes() kullanabiliriz.
df_categorical = df.select_dtypes(include = "object")
df_categorical.head(3)

In [ ]:
# Kategorik bir değişkendeki benzersiz değerleri gözlemlemek için unique() kullanabiliriz.
df_categorical["Status"].unique()

In [ ]:
# Kategorik değişkenlerdeki benzersiz değerlerin sayısını görebilmek için nunique() kullanabiliriz.
for col in df_categorical.columns:
    print(f'There are {df_categorical[col].nunique()} unique values in "{col}" categorical feature.\n')

In [ ]:
# Her bir kategorik değişken için toplam gözlem sayısını value_counts() ile buluyoruz.
for col in df_categorical.columns:
    print(f'{df_categorical[col].value_counts()}\n', 5*"*********")

In [ ]:
# Kategorik bir değişkende bulunan kategorik değerleri bir liste haline getirip sonra kullanabiliriz.
comp_categories = df["Type"].unique().tolist()[::-1]
comp_categories

In [ ]:
# Ordinal değişken nasıl oluşturulur?
from pandas.api.types import CategoricalDtype

df["Type"] = df["Type"].astype(CategoricalDtype(categories = comp_categories, ordered = True))

# ALIŞTIRMA: astype('category') ile CategoricalDtype arasında fark var mıdır? Varsa nedir? Araştırın.

In [ ]:
df["Type"].head()

**3.2 Kategorik Değişkenlerde Görselleştirme İşlemleri**

In [ ]:
# Kategorik bir değişkendeki kategorilerin gözlem sayılarını basitçe görselleştirelim.
df['Demographics'].value_counts().plot.barh();

In [ ]:
sns.countplot(data = df,
              y = "Demographics",
              hue = "Demographics",
              order = df["Demographics"].value_counts().index,
              palette = "Set1");

In [ ]:
sns.barplot(data = df, x = "Status", y = df.Status.index);

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Veriyle ilgili catplot
sns.catplot(data=df, x="Source", y="Score")

# X eksenindeki etiketleri 90 derece döndürmek
plt.xticks(rotation=90)

# Grafiği göster
plt.show()

In [ ]:
plt.figure(figsize = (15, 8))
sns.barplot(data = df,
            x = "Source",
            y = "Score",
            hue = "Demographics");
plt.xticks(rotation=90)

plt.show()

**3.3 Kategorik Gruplama (groupby) İşlemleri**

In [ ]:
df.head()

Soru: Hangi demografik grup, anime puanları açısından en yüksek ortalama değerlendirmeyi almıştır?

In [ ]:
df_usd = df.groupby(by = "Demographics")["Score"].mean().to_frame().reset_index().sort_values(by = "Score", ascending = False)

df_usd

**Soru:**  En yüksek ortalama anime puanına sahip olan demografik grup ve derecelendirme kombinasyonları nelerdir?

In [ ]:
df.groupby(by = ['Demographics', 'Rating'])['Score'].mean().to_frame().reset_index().sort_values(by = "Score", ascending = False)[:5]

In [ ]:
plt.figure(figsize = (12, 8))
plt.xticks(rotation = 90)
sns.barplot(data = df, x = "Demographics", y = "Score", hue = "Rating");

**Soru:** Belirli bir anime türü (Genre) için hangi izleyici kitlesi (Demographics) daha fazla kayıt sayısına sahiptir?

In [ ]:
print(df.columns)

In [ ]:
df_genre_demo = df.groupby(by="Genres")["Demographics"].value_counts().to_frame(name="Count").reset_index().sort_values(by="Count", ascending=False)
specific_genre = "Action"  # Örneğin, 'Action' türünü seçiyoruz
df_genre_demo[df_genre_demo['Genres'] == specific_genre]

In [ ]:
print(df_genre_demo.head())

In [ ]:
df_genre_demo = df.groupby(by="Genres")["Demographics"].value_counts().to_frame(name="Count").reset_index()

In [ ]:
sns.barplot(data=df_genre_demo, x="Genres", y="Count", hue="Demographics")

plt.xticks(rotation=90)

plt.show()

In [ ]:
print(df_genre_demo.columns)

In [ ]:
data = {
    "Genres": ["Action", "Drama", "Comedy"],
    "Demographics": ["Shonen", "Seinen", "Shonen"],
    "Count": [50, 30, 45]
}
df_genre_demo = pd.DataFrame(data)

sns.barplot(data=df_genre_demo, x="Genres", y="Count", hue="Demographics")
plt.show()

## 4. Sürekli Değişken Analizi

**4.1 Sürekli Değişkenlerin İncelenmesi**

In [ ]:
# Veri setindeki numerik kolonları seçmek için aşağıdaki yapıyı kullanabiliriz.
df_numerical = df.select_dtypes(include = ["float64", "int64"])
df_numerical.head()

In [ ]:
df_numerical.describe().T

In [ ]:
# Bir veri setindeki numerik kolonların istatistiksel bilgilerine erişmek için basitçe bu tarz bir fonksiyon yazabiliriz.

def give_stats(dataframe: pd.DataFrame) -> None:
    """Prints statistical information for numerical columns.

    Args:
        dataframe (pd.DataFrame): DataFrame object.
    
    Return:
        None
    
    """

    num_df = dataframe.select_dtypes(include = ["float", "int"])

    for col in num_df.columns:
        print(f"**********{col}**********")
        print(f"Mean value of {col} is {num_df[col].mean():.2f}")
        print(f"Std value of {col} is {num_df[col].std():.2f}")
        print(f"Max value of {col} is {num_df[col].max()}")
        print(f"Min value of {col} is {num_df[col].min()}")
        print(f"Count value of {col} is {num_df[col].count()}")
        print(f"Median value of {col} is {num_df[col].median()}\n")

In [ ]:
give_stats(dataframe = df)

In [ ]:
print(df.columns)

In [ ]:
# Numerik bir kolonun dağılımını görmek için histogram kullanırız.
sns.histplot(data = df, x = "Rating", kde = True, hue = "Demographics");

plt.xticks(rotation=90)

plt.show()

In [ ]:
# kdeplot ise ilgili numerik değişkenin yoğunluğunu gösterir.
sns.kdeplot(df['Scored_Users'], fill = True);

In [ ]:
df.replace([np.inf, -np.inf], np.nan, inplace=True)
df.dropna(inplace=True)

In [ ]:
zero_variance_categories = df.groupby("Demographics")["Scored_Users"].std() == 0
print(zero_variance_categories[zero_variance_categories].index)

In [ ]:
df = df[~df["Demographics"].isin(zero_variance_categories[zero_variance_categories].index)]

In [ ]:
print(df.isnull().sum())  # Eksik değer kontrolü
print(df.describe())      # Temel istatistikler

In [ ]:
sns.FacetGrid(data=df, hue="Demographics", height=7, xlim=(0, 400000)).map(
    sns.kdeplot, "Scored_Users", fill=True, warn_singular=False
).add_legend()

In [ ]:
# Catplot ile numerik bir değişkenin kategorik bazda ve kırılımdaki dağılımını görebiliriz.
plt.figure(figsize = (12, 8))
sns.catplot(data = df, x = "Demographics", y = "Scored_Users", hue = "Status", kind = "point");

plt.xticks(rotation=90)

plt.show()

In [ ]:
# Boxplot kategorik olarak numerik dağılımı gösterir, çeyreklik değerler ve IQR'a göre aykırı değerler de boxplot ile gözlemlenebilir.
sns.boxplot(data = df,
            x = "Demographics",
            y = "Scored_Users",
            hue = "Status");

plt.xticks(rotation=90)

plt.show()

In [ ]:
# Farklı numerik görselleştirme işlemleri için iris veri setini yükleyelim.
iris = sns.load_dataset(name = "iris")
iris[:3]

In [ ]:
# pairplot numerik değişkenler arasındaki saçılım ve yoğunluk ilişkilerini gösterir.
sns.pairplot(data = iris, hue = "species");

In [ ]:
# Scatter Plot, numerik değişkenler arasındaki saçılım ilişkisini gösterir.
sns.scatterplot(data = iris,
                x = "sepal_width",
                y = "sepal_length",
                hue = "species");

In [ ]:
# Heatmap ise corr() (korelasyon) methodu ile kullanıldığında numerik değerler arasındaki ilişki kuvvetini gösterir.
plt.figure(figsize = (10, 7))
sns.heatmap(iris[['sepal_length', 'sepal_width', 'petal_length', 'petal_width']].corr(), annot = True, cmap = "coolwarm");

In [ ]:
# Çizgi grafik ile numerik değişkenlerin tarihsel olarak değişimini gözlemleyebiliriz.
plt.figure(figsize = (10, 8))
sns.lineplot(data = df,
             x = "Ranked",
             y = "Scored_Users",
             hue = "Demographics");

## 5. Aykırı Değer Analizi (Outliers)

Aykırı değerlerin analizi de tıpkı eksik verilerde olduğu gibi hassasiyetle değerlendirilmelidir. Aykırı değerlerin varlığı veri setindeki dağılımları etkileyeceği için, aykırı değere sahip bir veri setiyle tahmin modeli oluşturduğumuzda modelimizin genellenebilirliğinin düşmesine sebep olacaktır.

Aykırı değerlerin değerlendirilmesi için sektörel bilgi, standart sapma yaklaşımı, Z-skoru, IQR yöntemi gibi yöntemler kullanılabilir. Biz burada IQR yöntemi ile basitçe bir düzeltme işlemi uygulayacağız.

In [ ]:
df = df_.copy()

In [ ]:
# Bir değişkendeki IQR'a göre aykırı gözlemleri boxplot kullanarak görselleştirelim.
plt.figure(figsize = (8, 6))
sns.boxplot(data = df,
            y = df["Scored_Users"],
            orient = "v");

In [ ]:
df_Scored_Users = df['Scored_Users']
df_Scored_Users

In [ ]:
# Quantile değerlerin belirlenmesi.
Q1 = df_Scored_Users.quantile(0.25)
Q3 = df_Scored_Users.quantile(0.75)

print(Q1)
print(Q3)

# IQR değerin belirlenmesi.
IQR = Q3-Q1
print(IQR)

# Alt ve üst sınırların belirlenmesi.
lower_fence = Q1 - 1.5*IQR
upper_fence = Q3 + 1.5*IQR

In [ ]:
# Upper_fence üzerinde kalan aykırı gözlemlerin index değerlerini, daha sonra kullanmak üzere bir değişkende tutabiliriz.
outlier_idx = df_Scored_Users[df_Scored_Users > upper_fence].index

outlier_idx

In [ ]:
df_Scored_Users[df_Scored_Users > upper_fence]

**5.1 Aykırı Gözlemlerin Silinmesi**

In [ ]:
df_del = df[~(df_Scored_Users > upper_fence)]

df_del

In [ ]:
plt.figure(figsize = (8, 6))
sns.boxplot(data = df_del,
            y = df_del["Scored_Users"],
            orient = "v");

**5.2 Aykırı Gözlemlerin Doldurulması**

**5.2.1 Ortalama Değer İle Doldurma**

In [ ]:
print(df.columns)

In [ ]:
df.loc[df_Scored_Users > upper_fence, "Scored_Users"] = df_Scored_Users.mean()

In [ ]:
df.loc[df_Scored_Users > upper_fence, "Scored_Users"]

In [ ]:
plt.figure(figsize = (8, 6))
sns.boxplot(data = df_del,
            y = df["Scored_Users"],
            orient = "v");

**5.2.2 Baskılama Yöntemi**

In [ ]:
df = df_.copy()

In [ ]:
df.loc[df_salary_in_usd > upper_fence, "Scored_Users"] = upper_fence

In [ ]:
plt.figure(figsize = (8, 6))
sns.boxplot(data = df_del,
            y = df["Scored_Users"],
            orient = "v");

**5.2.3 Local Outlier Factor**

In [ ]:
import numpy as np
import pandas as pd
from sklearn.neighbors import LocalOutlierFactor

In [ ]:
# Sayısal sütunları seçin
df_numeric = df.select_dtypes(include=[float, int])

# Veriyi inceleyin
print(df_numeric.head())

In [ ]:
# Veri setinde 'Duration_Minutes' sütununu kontrol edin
print(df['Duration_Minutes'].head())

# 'Duration_Minutes' sütununun sayısal veriler olup olmadığını kontrol edin
print(df['Duration_Minutes'].dtype)  # Bu 'int' veya 'float' olmalı

In [ ]:
# Eğer eksik değerler varsa, onları ortalama ile doldurabilirsiniz
df['Duration_Minutes'].fillna(df['Duration_Minutes'].mean(), inplace=True)

In [ ]:
# 'Duration_Minutes' sütunundaki NaN değerlerini ortalama ile dolduruyoruz
df['Duration_Minutes'].fillna(df['Duration_Minutes'].mean(), inplace=True)

In [ ]:
# Tüm sayısal sütunlardaki NaN değerlerini ortalama ile doldur
df_numeric = df.select_dtypes(include=[float, int])  # Sayısal sütunları seç
df_numeric.fillna(df_numeric.mean(), inplace=True)

In [ ]:
# Veriden eksik değerler içeren satırları çıkarıyoruz
df_numeric = df.select_dtypes(include=[float, int])  # Sayısal sütunları seç
df_numeric.dropna(inplace=True)

In [ ]:
import pandas as pd
from sklearn.neighbors import LocalOutlierFactor

# Verinizi yükleyin ve sayısal sütunları seçin
# Örneğin df, veri setiniz
df_numeric = df.select_dtypes(include=['float64', 'int64'])

# Eksik değerleri, her sütunun ortalama değeriyle dolduruyoruz
df_numeric.fillna(df_numeric.mean(), inplace=True)

# LOF modelini oluşturuyoruz
lof = LocalOutlierFactor(n_neighbors=20, metric='euclidean')

# Anomali puanlarını hesaplıyoruz
outlier_scores = lof.fit_predict(df_numeric)

# LOF puanlarını veriye ekliyoruz
df['LOF_Score'] = lof.negative_outlier_factor_

# Anomali tespiti: -1 anomalidir, 1 ise normaldir
df['Outlier'] = outlier_scores == -1

# Eğer 'anime_title' sütunu mevcutsa, sadece onu da ekleyebiliriz:
if 'anime_title' in df.columns:
    print(df[['anime_title', 'Duration_Minutes', 'LOF_Score', 'Outlier']].head())
else:
    print(df[['Duration_Minutes', 'LOF_Score', 'Outlier']].head())

## 6. Feature Engineering

Bu bölümde veri setindeki özellikleri kullanarak ne tür özellikler türetebileceğimize ilişkin örnekler göreceğiz.

In [ ]:
df.head()

In [ ]:
# Anime'nin uzunluğunu (süre) göz önünde bulundurarak, anime'leri kısa, orta ve uzun süreli kategorilere ayırabilirsiniz.

def anime_duration_group(duration):
    if duration < 12:
        return 'Short'
    elif 12 <= duration < 24:
        return 'Medium'
    else:
        return 'Long'

df['duration_group'] = df['Duration_Minutes'].apply(anime_duration_group)
df.head(3)

**Açıklama:** Anime süreleri, anime türlerinin (örneğin, kısa filmler veya uzun seriler) farklı kategorilere ayrılmasına yardımcı olabilir. Bu, kullanıcılara daha kısa veya daha uzun anime önerileri yapmak için yararlı olabilir.

In [ ]:
# Rating sütununu sayısal değerlere dönüştürme
df['Rating'] = pd.to_numeric(df['Rating'], errors='coerce')  # 'coerce' hatalı değerleri NaN yapar

# Rating'in sayısal hale gelip gelmediğini kontrol edelim
print(df['Rating'].head())

In [ ]:
# Eksik değerleri kontrol etme
print(df['Rating'].isnull().sum())

# Eksik değerleri ortalama ile doldurma (veya başka bir yöntem seçebilirsiniz)
df['Rating'] = df['Rating'].fillna(df['Rating'].mean())

In [ ]:
# Anime'nin IMDB veya MAL (MyAnimeList) puanlarına göre izleyici kitlesine yönelik segmentasyonlar yapılabilir.
def rating_category(rating):
    if rating < 5:
        return 'Low'
    elif 5 <= rating < 7:
        return 'Medium'
    else:
        return 'High'

df['rating_category'] = df['Rating'].apply(rating_category)
df.head(3)

**Sonuç**

Bu projede, anime veri setini kullanarak çeşitli analizler ve özellik mühendisliği teknikleri uyguladık. Verinin içeriği hakkında daha derin bir anlayış geliştirmek ve anlamlı segmentasyonlar yapmak amacıyla farklı özellikler türettik. Ana hedef, anime verileri üzerinden anlamlı kategoriler oluşturmak ve bu kategorileri kullanarak izleyici kitlesine yönelik öngörülerde bulunmaktı.

**Özellikle aşağıdaki analizler ve sonuçlar elde edildi:**

* Rating Kategorisi Segmentasyonu: Anime'nin IMDB veya MyAnimeList (MAL) puanlarına göre izleyici kitlesine yönelik segmentasyonlar yapıldı. Rating sütununa dayalı olarak, düşük, orta ve yüksek puanlı animeler kategorize edildi. Bu segmentasyon, izleyicilerin tercihlerine göre anime seçimlerini anlamada yardımcı olabilir.

* Outlier Tespiti: Z-Score ve LOF (Local Outlier Factor) gibi yöntemler kullanarak, veri setinde yer alan anormal değerler (outliers) tespit edildi. Bu, verinin doğruluğunu artırmak ve analizin güvenilirliğini sağlamak için önemlidir.

* Anime Süresi (Duration) Segmentasyonu: Anime'nin süresine göre segmentasyon yapılarak, kısa, orta ve uzun süreli animeler kategorize edildi. Bu özellik, anime izleyicilerinin zaman tercihleri hakkında bilgi edinmemize olanak tanır.

* Yaş Kategorisi (Age) Segmentasyonu: Anime izleyicilerinin yaş grubuna göre segmentasyonlar yapıldı. Bu segmentasyon, animelerin belirli yaş gruplarına hitap ettiğini anlamamıza yardımcı olabilir.

* Özellik Mühendisliği: Bu projede, anime veri setinde yer alan çeşitli sayısal ve kategorik veriler kullanılarak anlamlı özellikler türetildi. Özellikle anime süresi, puan, tür gibi özellikler üzerinde yapılan kategorize işlemleri, daha anlamlı segmentasyonların elde edilmesine katkı sağladı.


**Gelecek Çalışmalar ve İyileştirmeler**

* Bu proje, anime veri setine dayalı analizlerde başlangıç seviyesinde önemli çıkarımlar sağlamıştır. Ancak, gelecekte daha karmaşık modellerin ve algoritmaların uygulanması, anime verileri üzerinden daha derinlemesine öngörüler yapabilmeyi mümkün kılacaktır. Ayrıca, verilerin temizlenmesi ve işlenmesi konusunda daha fazla iyileştirme yapılabilir. Örneğin, daha sofistike outlier tespit yöntemlerinin kullanılması ve eksik veriler için daha gelişmiş imputation tekniklerinin uygulanması faydalı olabilir.

* Son olarak, projede yapılan segmentasyonlar ve özellik mühendisliği yöntemleri, anime dünyasında izleyicilerin tercihlerine yönelik daha etkili öneri sistemleri geliştirilmesine katkı sağlayabilir.